不带参数的层

In [3]:
import torch
import torch.nn.functional as F 
from torch import nn 

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        """实现前向传播功能"""
        return X-X.mean()

In [4]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [6]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [7]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(0., grad_fn=<MeanBackward0>)

带参数的层

In [8]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units, ))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data 
        return F.relu(linear)

In [ ]:
linear = MyLinear(5, 3)   # 要求输入为5， 输出为3
linear.weight

Parameter containing:
tensor([[ 0.9106,  0.1664, -0.0883],
        [-0.1092,  0.0313, -1.0162],
        [-0.3709,  1.3383,  1.4982],
        [ 0.3242, -1.8513,  0.4223],
        [ 0.9668,  1.1432, -0.5760]], requires_grad=True)

In [10]:
linear(torch.rand(2, 5))

tensor([[1.5194, 0.0000, 0.2807],
        [1.3310, 0.0000, 0.0095]])

In [11]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[2.3750],
        [5.8235]])

练习

设计一个接受输入并计算张量降维的层, 它返回yk = sum(wijk * xi * xj)

In [ ]:
class downLayer(nn.Module):
    """ 
    设计一个接受输入并计算张量降维的层, 它返回yk = sum(wijk * xi * xj)
    这里只设置了接受单个样本
    """
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, in_units, units))
    
    def forward(self, X):
        """
        返回yk = sum(wijk * xi * xj)

        Parameter
        ----------
            X : 输入数据，(in_units, )
        """
        X = X.unsqueeze(1) # unqueeze在指定位置增添一个维度, 即在第一维插入一个维度 (in_units, 1)
        X_outer = torch.mm(X, X.t())  # (in_units, in_units)
        y = torch.einsum('ij,jkl->kl', X_outer, self.weight)
        # einsum 爱因斯坦求和约定
        # 对输入中重复出现的标签j执行矩阵乘法
        # 在输出中重复出现的标签k,l表示保留该维度
        # 在输出中未重复出现(只在输入中出现)的标签i表示对该维度求和
        # X_outer (i, j)   self.weight(j, k, l)
        # 对X_outer的每个元素X[i,j]和self.weight的切片w[j,:,：]执行元素级乘法，得到temp[i, k, l]
        # 对i维度求和，最终输出y[k,l]
        
        return y.squeeze() # squeeze 移除所有维度大小为1的维度，使张量的维度数减少


In [15]:
layer1 = downLayer(5, 2)
layer1(torch.randn(5))

tensor([[-1.2253,  4.1714],
        [ 4.3161, -0.4322],
        [-1.2593,  0.4844],
        [-0.9717, -1.1399],
        [ 0.4264, -2.5725]], grad_fn=<SqueezeBackward0>)

设计一个返回输入数据的傅里叶系数前半部分的层

In [19]:
class FourierCoefficientLayer(nn.Module):
    """ 
    返回输入数据的傅里叶系数前半部分的自定义层
    支持1D、2D、3D的信号, 自动处理批量维度和通道维度
    """
    def __init__(self, signal_dim=1):
        """ 
        初始化傅里叶系数层

        Parameter
        ----------
            signal_dim : 信号维度, 1表示1D信号, 2表示2D信号, 3表示3D信号
        """
        super().__init__()
        self.signal_dim = signal_dim

    def forward(self, X):
        """ 
        前向传播计算 

        Parameter
        ----------
            X : 输入张量, 形状应为(batch_size, channels, *signal_shape), signal_shape的长度应与signal_dim一致
        
        Return
        ----------
            傅里叶系数的前半部分, 形状根据输入信号维度不同而不同
        """
        # 检查输入维度是否匹配
        if len(X.shape)-2 != self.signal_dim:
            raise ValueError(f"输入张量维度应为 (batch, channels, {self.signal_dim}信号)")

        # 根据信号维度选择傅里叶变换函数
        if self.signal_dim == 1:
            # 1D信号: (batch, channels, length)
            fft_result = torch.fft.rfft(X, dim=-1)
        elif self.signal_dim == 2:
            # 2D信号: (batch, channels, height, width)
            fft_result = torch.fft.rff2(X, dim=(-2, -1))
        elif self.signal_dim == 3:
            # 3D信号: (batch, channels, depth, height, width)
            fft_result = torch.fft.rfftn(X, dim=(-3, -2, -1))
        else:
            raise ValueError("signal_dim必须为1、2或3")
        
        return fft_result # 返回傅里叶系数的前半部分

In [20]:
x_1d = FourierCoefficientLayer(1)
x_1d(torch.randn(4, 3, 4))

tensor([[[ 2.8320+0.0000j,  0.9570+0.3435j,  0.9783+0.0000j],
         [ 2.9490+0.0000j,  0.2558-0.8542j,  1.8705+0.0000j],
         [-1.4231+0.0000j,  0.5572-0.1542j,  1.1135+0.0000j]],

        [[ 1.7841+0.0000j,  0.4297+0.4376j,  0.5546+0.0000j],
         [-1.4070+0.0000j,  1.7606+0.2316j, -2.7826+0.0000j],
         [ 0.7130+0.0000j,  1.1661-0.9240j, -2.1249+0.0000j]],

        [[-3.1791+0.0000j, -0.4591-0.4848j, -0.2474+0.0000j],
         [-0.2449+0.0000j, -0.0964-0.1509j, -3.1830+0.0000j],
         [ 0.5365+0.0000j, -0.9598+0.8740j, -0.3699+0.0000j]],

        [[ 3.6898+0.0000j, -0.2435+1.8472j,  5.7723+0.0000j],
         [ 2.2252+0.0000j, -1.4074+0.6666j, -1.0055+0.0000j],
         [-1.8485+0.0000j, -0.1693+1.3783j,  0.5546+0.0000j]]])